In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pickle

# life path functions

<img src='individual_dao_life_cycle.png' width="400" height="800">


In [2]:
# models and parameters
def daocost_change(timestep,state_history,**params):
    return curve_generator(timestep,**params)
def daoincome_change(timestep,state_history,**params): # right now, same function as cost
    return curve_generator(timestep,**params)        
    
def nearfund2dao_change(timestep,state_history,**params): 
    if params['curvetype']=='constant':
        a = params['amount']
        return a,'L1'
    if params['curvetype']=='step':
        a1 = params['L1amount']
        a2 = params['L2amount']
        league_policy = params['league_policy']
        leaguenow = league_policy(state_history)
        if leaguenow=='L2':            
            return a2,'L2'
        else:
            return a1,'L1'   
    if params['curvetype']=='survive_step':
        a1 = params['L1amount']
        a2 = params['L2amount']
        league_policy = params['league_policy']
        leaguenow = league_policy(state_history)
        if (leaguenow=='L2'):
            is_indanger = (state_history[-1]['cost_step']-state_history[-1]['income_step']) > state_history[-1]['treasury']*1.2

            if (is_indanger):            
                return a2,'L2'
            else:
                return a1,'L2'
        else:
            return a1,'L1'

In [3]:
def curve_generator(timestep,**params):
    if params['curvetype']=='saturate':
        a = params['speed']
        b = params['maximum']
        c = params['start']
        assert b>c ,'minimum should be bigger than the start'
        return (1/(1 + np.exp(-a*timestep)) - .5) *(b-c)*2 + c
    if params['curvetype']=='noisy_saturate':
        a = params['speed']
        b = params['maximum']
        c = params['start']
        d = params['noise']
        assert b>c ,'minimum should be bigger than the start'
        return max((1/(1 + np.exp(-a*timestep)) - .5) *(b-c)*2 + c + np.random.randn()*d,0)
    if params['curvetype']=='logistic':
        a = params['speed']
        b = params['maximum']
        c = params['start']
        t0 = params['midpoint'] # when the exponential growth switch to linear / sublinear
        assert b>c ,'minimum should be bigger than the start'
        return (1/(1 + np.exp(-a*(timestep-t0))) - 1/(1+np.exp(a*t0))) *(b-c)/(1-1/(1+np.exp(a*t0))) +c
    if params['curvetype']=='noisy_logistic':
        a = params['speed']
        b = params['maximum']
        c = params['start']
        t0 = params['midpoint'] # when the exponential growth switch to linear / sublinear
        d = params['noise']
        assert b>c ,'minimum should be bigger than the start'
        return max((1/(1 + np.exp(-a*(timestep-t0))) - 1/(1+np.exp(a*t0))) *(b-c)/(1-1/(1+np.exp(a*t0))) +c+ np.random.randn()*d,0)
    if params['curvetype']=='fluctuation': # gaussian noise added on top of constant
        a = params['mean']
        b = params['noise']
        return max(a + np.random.randn()*b,0)
        
        

In [4]:
# the state update model
def daovariable_change(daotreasury,costmodel,incomemodel,fundmodel,timestep,state_history):
    cost_T = daocost_change(timestep,state_history,**costmodel)
    income_T = daoincome_change(timestep,state_history,**incomemodel) 
    fund_T,league_T=nearfund2dao_change(timestep,state_history,**fundmodel)

    netgain_T = income_T + fund_T - cost_T
    treasurey_T = daotreasury + netgain_T
    return {'treasury':treasurey_T,'cost_step':cost_T,'income_step':income_T,'nearfund_step':fund_T,'netgain_step':netgain_T,'league':league_T}
    

In [5]:
def runsimu(Nstep,ini_treasury,models):
    # simulation engine, putting things together
   
    #initialize
    treasury_t = ini_treasury
    var_list = [{'treasury':treasury_t,'cost_step':np.nan,'income_step':np.nan,'netgain_step':np.nan,'nearfund_step':np.nan}]
    for t in range(Nstep):
        var_t = daovariable_change(treasury_t,models['costmodel'],models['incomemodel'],models['fundmodel'],t,var_list)
        var_list.append(var_t)
        treasury_t = var_t['treasury']
        if treasury_t < 0: # the DAO "dead", so just stop simulation
            var_list+=[{'treasury':np.nan,'cost_step':np.nan,'income_step':np.nan,'netgain_step':np.nan,'nearfund_step':np.nan} for k in range(Nstep-t-1)]
            break
    
    simu_df = pd.DataFrame(var_list)
    simu_df.index.rename('Time (month)',inplace=True)
    return simu_df


# Individual DAO's life path

## story 1: a small dao sustained by near funds

In [10]:
fundmodel = {'curvetype':'constant','amount':5}

costmodel = {'curvetype':'fluctuation','mean':7,'noise':1}
incomemodel = {'curvetype':'fluctuation','mean':2,'noise':1}

daotreasury = 2 # initial treasury for every dao

# start simulation
Nstep = 12
var_df = runsimu(Nstep,daotreasury,{'costmodel':costmodel,'incomemodel':incomemodel,'fundmodel':fundmodel})
px.line(var_df,y=['treasury'])

In [11]:
px.line(var_df,y=['cost_step','income_step','nearfund_step'])

## story 2: a dao failed to sustain

In [24]:
fundmodel = {'curvetype':'constant','amount':5}

costmodel = {'curvetype':'fluctuation','mean':6,'noise':2}
incomemodel = {'curvetype':'fluctuation','mean':2,'noise':1}

daotreasury = 2 # initial treasury for every dao

# start simulation
np.random.seed(121)
Nstep = 12
var_df = runsimu(Nstep,daotreasury,{'costmodel':costmodel,'incomemodel':incomemodel,'fundmodel':fundmodel})
px.line(var_df,y=['treasury'])

In [25]:
px.line(var_df,y=['cost_step','income_step','nearfund_step'])

## story 3: a dao with big growth

In [30]:
fundmodel = {'curvetype':'constant','amount':5}

costmodel = {'curvetype':'noisy_saturate','noise':2,'maximum':20,'speed':.6,'start':5}
incomemodel = {'curvetype':'noisy_saturate','noise':3,'maximum':20,'speed':.4,'start':2}

daotreasury = 2 # initial treasury for every dao

# start simulation
Nstep = 24
var_df = runsimu(Nstep,daotreasury,{'costmodel':costmodel,'incomemodel':incomemodel,'fundmodel':fundmodel})
px.line(var_df,y=['treasury'])

In [31]:
px.line(var_df,y=['cost_step','income_step','nearfund_step'])

# simulate a group of daos

**30%:** easily fail

**60%:** sustain

**10%:** great success!


In [33]:
# System simulation
def runsyssimu(Nstep,daolist,fundmodel):
    np.random.seed(1)
    sysvar_steps = [{'totaltreasury':0,'ndao':0,'steptransaction':0,'stepnearfund':0} for k in range(Nstep)]
    daosimu_list =[]
    for daomodel in daolist:        
        var_list = []
        #initialize
        treasury_t = daomodel['ini_treasury']
        for t in range(Nstep):
            var_t = daovariable_change(treasury_t,daomodel['costmodel'],daomodel['incomemodel'],fundmodel,t,var_list)
            var_list.append(var_t)
            treasury_t = var_t['treasury']
            if treasury_t < 0: # the DAO "dead", so just stop simulation
                var_list+=[{'treasury':np.nan,'cost_step':np.nan,'income_step':np.nan,'netgain_step':np.nan,'nearfund_step':np.nan} for k in range(Nstep-t-1)]
                break
            else:
                sysvar_steps[t]['totaltreasury'] += treasury_t
                sysvar_steps[t]['ndao'] +=1
                sysvar_steps[t]['steptransaction'] += var_t['income_step']+var_t['cost_step']                
                sysvar_steps[t]['stepnearfund'] +=var_t['nearfund_step']

        daosimu_df = pd.DataFrame(var_list)
        daosimu_list.append(daosimu_df)
    sysvar_df = pd.DataFrame(sysvar_steps)
    return sysvar_df,daosimu_list

In [110]:
# Initialize funding policy
fundmodel = {'curvetype':'constant','amount':5}
daotreasury = 2 # initial treasury for every dao

# Initialize player distribution
daolist = []
for k in range(20): # daos that easily fail
    daocostmodel = {'curvetype':'noisy_saturate','maximum':15+np.random.randint(3),'speed':.2+np.random.rand()*8,'start':1+2*np.random.rand(),'noise':1}
    daoincomemodel = {'curvetype':'noisy_saturate','maximum':10,'speed':0.2,'start':1,'noise':1}
    daolist.append({'incomemodel':daoincomemodel,'costmodel':daocostmodel,'ini_treasury':daotreasury})

for k in range(60): # self-sustained models
    daocostmodel = {'curvetype':'noisy_logistic','maximum':6,'speed':.8+np.random.rand()*8,'start':4,'midpoint':10,'noise':1}
    daoincomemodel = {'curvetype':'noisy_logistic','maximum':2.5+np.random.randint(4),'speed':1.2,'start':2,'midpoint':11,'noise':1}
    daolist.append({'incomemodel':daoincomemodel,'costmodel':daocostmodel,'ini_treasury':daotreasury})

for k in range(20): # crazy successful models
    daocostmodel = {'curvetype':'noisy_logistic','maximum':50,'speed':.7+np.random.rand()*.4,'start':4+np.random.randint(5),'midpoint':10+np.random.randint(4),'noise':2}
    daoincomemodel = {'curvetype':'noisy_logistic','maximum':65+np.random.randn()*12,'speed':.9,'start':4,'midpoint':12,'noise':2}
    
    daolist.append({'incomemodel':daoincomemodel,'costmodel':daocostmodel,'ini_treasury':daotreasury})

In [111]:
# let's run it!
Nstep=24
sysvar_df,daosimu_list = runsyssimu(Nstep,daolist,fundmodel)


In [112]:
pickle.dump([daolist,sysvar_df,daosimu_list],open('daolist_syssimu.p','wb'))

## system health metrics visualized

### number of active DAOs

In [113]:
px.line(sysvar_df,y=['ndao'],title='Number of active DAOs')

as expected...about 30% of daos failed

### Total transaction (cost+income) happened at each step

In [114]:
px.line(sysvar_df,y=['steptransaction'],title='Platform-sum transaction each step')



Initially when daos die out, the system actually has less total transaction. Yet gradually with the growth of some daos, the total transaction gets to a higher level than initial

## demonstrate the diversity of this system

In [115]:
dao_laststage = []
lastnstep=3
for k,ksimu in enumerate(daosimu_list):
    transaction = (sum(ksimu.iloc[-lastnstep:]['cost_step']) + sum(ksimu.iloc[-lastnstep:]['income_step']))/lastnstep
    treasury = ksimu.iloc[-lastnstep:]['treasury'].mean()
    dao_laststage.append({'kdao':k,'av_transaction':transaction,'av_dao_treasury':treasury})
dao_laststage = pd.DataFrame(dao_laststage)

### total transaction of a dao at the stable stage (last 3 time steps)

In [116]:
fig=px.pie(dao_laststage,names='kdao',values='av_transaction')
fig.update_traces(textinfo='none')


In [117]:
px.histogram(dao_laststage,x='av_dao_treasury',width=800,height=500)

Most daos are relatively small in terms of their treasury size and average transaction. Some daos are quite large whales though.

Correspondingly, there's a long tail distribution of treasury size.

# League policy

**motivation**: for the dead daos that could have survived and shown huge growth potential

In [101]:
import plotly.io as pio
pio.renderers.default='notebook'

In [120]:
var_df=daosimu_list[92]
px.line(var_df,y=['cost_step','income_step','nearfund_step'])

for daos like this, they are showing promising growth of income and cost, but just due to the gap of cost and income, they can't survive...

**League policy**:

- 
-

In [121]:
def league_incomegrowthrate(state_history):
    if len(state_history)<4:
        return 'L1'
    if state_history[-1]['league']=='L2': #once being in L2, will 
        return 'L2' # policy assumes no downgrade
    sh_df = pd.DataFrame(state_history)
    inc1 = sh_df.iloc[-3:]['income_step']
    inc2 = sh_df.iloc[-4:-1]['income_step']
    #breakpoint()
    incchange = inc1.values-inc2.values
    if sum(incchange<=0):
        return 'L1'
    else:
        return 'L2'

In [122]:
Nstep=24

fundmodel = {'curvetype':'survive_step','L1amount':5,'L2amount':20,'league_policy':league_incomegrowthrate}
# run simu!
sysvar_lg_df,daosimu_lg_list = runsyssimu(Nstep,daolist,fundmodel)

In [123]:
px.line(sysvar_lg_df,y=['ndao'],title='Number of active DAOs')

## contrast with or without league funding

In [124]:
cprsimu_df = pd.concat([sysvar_df, sysvar_lg_df.add_suffix('_league')],axis=1)
cprsimu_df.index.rename('time (month)',inplace=True)
px.line(cprsimu_df,y=['ndao','ndao_league'])

not surprising, more daos survived

In [125]:
px.line(cprsimu_df,y=['steptransaction','steptransaction_league'])

no surprise: more overall transaction happening in the system

In [126]:
px.line(cprsimu_df,y=['steptransaction','steptransaction_league','stepnearfund','stepnearfund_league'])

the benefit of saving some daos overweights the amount of increased funding that near fundation needs to give out. Thus, this policy is an efficient way for improving system health.

# think about non-financial impact

<img src='individual_impact.png' width="400" height="800">


In [127]:
def daoimpact(state_history,**params):
    impactfunc = params['func']
    impact = impactfunc(state_history,**params)    
    return impact

def transaction2impact(state_history,**params):
    # simple impact function
    # assuming a linear relationship between log total transaction and average impact. using log to close the gap between huge daos and smaller ones
    noise = params['noise']
    const = params['const'] 
    recent_trans = state_history[-1]['cost_step']+state_history[-1]['income_step']
    impact = max(const + np.log2(recent_trans) + np.random.randn()*noise,0.01)
    return impact

In [128]:
# get system impact from current simulation
impmodel = {'func':transaction2impact,'noise':.5}

system_impact = np.zeros(Nstep)
for k,kdao in enumerate(daosimu_lg_list):
    statehist = kdao.to_dict('records')
    daoimpmodel = impmodel.copy()
    daoimpmodel['const'] = np.random.rand()*6 # impact constant for this dao. bigger means bigger average impact
 
    implist = []
    for kt in range(Nstep):
        imp_t = daoimpact(statehist[:kt+1],**daoimpmodel)
        implist.append(imp_t)
        if not np.isnan(imp_t):
            system_impact[kt]+=imp_t
    kdao['impact_step']=implist
sysvar_lg_df['total_impact'] = system_impact

In [130]:
px.line(daosimu_lg_list[99],y=['impact_step'],title='Impact growth for an example DAO')

yes, when a dao grows bigger the impact will grow a bit bigger

In [131]:
def getdao_laststage(daosim_list):
    dao_laststage = []
    lastnstep=3
    for k,ksimu in enumerate(daosim_list):
        transaction = (sum(ksimu.iloc[-lastnstep:]['cost_step']) + sum(ksimu.iloc[-lastnstep:]['income_step']))/lastnstep
        treasury = ksimu.iloc[-lastnstep:]['treasury'].mean()
        income=ksimu.iloc[-lastnstep:]['income_step'].mean()
        cost=ksimu.iloc[-lastnstep:]['cost_step'].mean()
        impact = ksimu.iloc[-lastnstep:]['impact_step'].mean()
        dao_laststage.append({'kdao':k,'av_transaction':transaction,'av_dao_treasury':treasury,'av_income':income,'av_cost':cost,'av_impact':impact})
        
    dao_laststage = pd.DataFrame(dao_laststage)
    return dao_laststage

In [132]:
dao_lg_laststage = getdao_laststage(daosimu_lg_list)

## impact-based system health metrics

### basic check of the impact measurement: smaller daos can make big impact too

In [134]:
px.scatter(dao_lg_laststage,x='av_transaction',y='av_impact',title='Impact V.S. transaction size')

With the current impact model + parameter setting, the daos with higher economic impact (measured by average transaction) generally have higher non-economic impact too, but it's not neccesary -- daos with much less transaction could still have quite high impacts

## (non-)growth of system impact

In [135]:
px.line(sysvar_lg_df,y='total_impact')

So it's sad that because the league system only focused on the economic value of a dao, some daos that could have the potential of generating good impacts died. Then overall, we don't see a great growth of system impact. This is why there should be also special funding support for daos that generate good impact but not with great economic perspect.

## Introduce impact-based reward

<img src='Cadcad_impact_Reward.png' width="800" height="800">
